In [10]:
import chess as chess
import chess.engine as engine
import random
import numpy as np
import math
import tensorflow.keras.models as models
import tensorflow.keras.layers as layers
import tensorflow.keras.utils as utils
import tensorflow.keras.optimizers as optimizers

In [11]:
def random_board():
    board = chess.Board()
    depth = random.randrange(0, 300)
    for i in range(depth):
        all_moves = list(board.legal_moves)
        rand_move = random.choice(all_moves)
        board.push(rand_move)
        print(board)
        if board.is_game_over():
            break
    return board 

def sfrating(board, depth, boolean): #boolean true if checking whites score, false if checking blacks score
    stockfish = engine.SimpleEngine.popen_uci('ChessAI/stockfish-mac-master/Chess')
    result = stockfish.analyse(board, engine.Limit(depth=depth))
    if boolean:
        return result['score'].white().score()
    else:
        return result['score'].black().score()
    
def index(spot): #1, 63, etc, horizontal-sequenced
    return (8*(spot[1]-1)) + ord(spot[0].upper()-65) 

def three_d_ify(cb): #returns 14 states of board, an appearance matrix of every piece for both sides (6*2=12) and 2 more states with all that each side threatens
    super_board = np.zeros((14, 8, 8), dtype=np.int8) #simplified because max is 64
    
    for piece in chess.PIECE_TYPES:
        for sq in cb.pieces(piece, chess.WHITE):
            p_index = np.unravel_index(sq, (8, 8))
            super_board[piece-1][p_index[0]][p_index[1]] = 1 #targets white pieces of type
            
        for sq in cb.pieces(piece, chess.BLACK):
            p_index = np.unravel_index(sq, (8, 8))
            super_board[piece+5][p_index[0]][p_index[1]] = 1 #targets black pieces of type
            
    cb.turn = chess.WHITE
    for vmoves in cb.legal_moves:
        i = index(vmoves.to_square)
        super_board[12][i-math.floor((i)/8)*8][math.floor((i)/8)] = 1
        
    cb.turn = chess.BLACK
    for vmoves in cb.legal_moves:
        i = index(vmoves.to_square)
        super_board[13][i-math.floor((i)/8)*8][math.floor((i)/8)] = 1
        

In [13]:
def make_model(conv_layers, conv_neurons, dense_layers, dense_neurons): #i like to call conv filters neurons
    super_board = layers.Input(shape=(14, 8, 8))
    x = super_board
    
    for i in range(conv_layers):
        x = layers.Conv2D(filters=conv_neurons, kernel_size=3, padding='same', activation='relu')(x)
    x = layers.Flatten()(x)
    
    for i in range(dense_layers):
        x = layers.Dense(units=dense_neurons, activation='relu')(x)
    x = layers.Dense(units=1, activation='sigmoid')(x)
    
    return models.Model(inputs=super_board, outputs=x)


AttributeError: module 'pydot' has no attribute 'InvocationException'

In [15]:
def minimax(board, depth, maximizing, alpha, beta, model):
    if depth == 0 | board.is_game_over(): 
        return evaluate(board, model)
    
    if maximizing:
        max_move = -np.inf
        for move in board.legal_moves:
            newb = board.push(move)
            eval = minimax(newb, depth-1, False, alpha, beta, model)
            board.pop()
            max_move = max(max_move, eval)
            alpha = max(alpha, eval)
            if alpha >= beta:
                return
        return max_move
    
    else:
        min_move = np.inf
        for move in board.legal_moves:
            newb = board.push(move)
            eval = minimax(newb, depth-1, True, alpha, beta, model)
            board.pop()
            min_move = min(min_move, eval)
            beta = min(beta, eval)
            if alpha >= beta:
                return
        return min_move    

def evaluate(board, model):
    super_board = three_d_ify(board)
    model.predict(super_board)
            
def get_move(board, depth, model):
    m = None
    
    if board.turn == chess.WHITE:
        max_eval = -np.inf
        for move in board.legal_moves:
            board.push(move)
            eval = minimax(board, depth, True, -np.inf, np.inf, model)
            board.pop()
            if eval > max_eval:
                max_eval = eval
                m = move
    
    else:
        min_eval = np.inf
        for move in board.legal_moves:
            board.push(move)
            eval = minimax(board, depth, False, -np.inf, np.inf, model)
            board.pop()
            if eval < min_eval:
                min_eval = eval
                m = move
                
    return m            